<a href="https://colab.research.google.com/github/sarawoldeyohannes/BigData/blob/main/BigData_assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q pyspark

!wget -q 'https://drive.google.com/uc?export=download&id=1cyEVCdW2V1lYtv1eYImFVvR-HsqenSY4' -O 'W25Sample.txt'

In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

conf = SparkConf().setAppName('SparkWordCount')
sc = SparkContext.getOrCreate(conf = conf)

sqlContext = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
#word count
input_file = sc.textFile('W25Sample.txt')

input_file.collect()[:10]

['The Project Gutenberg EBook of The Complete Works of William Shakespeare, by',
 'William Shakespeare',
 '',
 'This eBook is for the use of anyone anywhere at no cost and with',
 'almost no restrictions whatsoever. You may copy it, give it away or',
 're-use it under the terms of the Project Gutenberg License included',
 'with this eBook or online at www.gutenberg.org',
 '',
 '** This is a COPYRIGHTED Project Gutenberg eBook, Details Below **',
 '** Please follow the copyright guidelines in this file. **']

In [ ]:
flatMap = input_file.map(lambda line: line.split())

flatMap.collect()[:2]

[['The',
  'Project',
  'Gutenberg',
  'EBook',
  'of',
  'The',
  'Complete',
  'Works',
  'of',
  'William',
  'Shakespeare,',
  'by'],
 ['William', 'Shakespeare']]

In [ ]:
def sixWords(words):
  res=[]
  n=len(words)
  if n< 6:
    return res
  for i in range(n-5):
    seq=' '.join(words[i:i+6])
    res.append(seq)
  return res
sequences = flatMap.flatMap(sixWords)

In [ ]:
keyPairs = sequences.map(lambda word: (word, 1))

keyPairs.collect()[:10]

[('The Project Gutenberg EBook of The', 1),
 ('Project Gutenberg EBook of The Complete', 1),
 ('Gutenberg EBook of The Complete Works', 1),
 ('EBook of The Complete Works of', 1),
 ('of The Complete Works of William', 1),
 ('The Complete Works of William Shakespeare,', 1),
 ('Complete Works of William Shakespeare, by', 1),
 ('This eBook is for the use', 1),
 ('eBook is for the use of', 1),
 ('is for the use of anyone', 1)]

In [ ]:
countKeys = keyPairs.reduceByKey(lambda a, b: a + b)

countKeys.collect()[:20]

[('EBook of The Complete Works of', 2),
 ('Complete Works of William Shakespeare, by', 1),
 ('eBook is for the use of', 2),
 ('for the use of anyone anywhere', 2),
 ('the use of anyone anywhere at', 2),
 ('of anyone anywhere at no cost', 2),
 ('anyone anywhere at no cost and', 2),
 ('You may copy it, give it', 2),
 ('may copy it, give it away', 2),
 ('copy it, give it away or', 2),
 ('re-use it under the terms of', 2),
 ('it under the terms of the', 2),
 ('under the terms of the Project', 2),
 ('terms of the Project Gutenberg License', 2),
 ('of the Project Gutenberg License included', 2),
 ('** This is a COPYRIGHTED Project', 1),
 ('COPYRIGHTED Project Gutenberg eBook, Details Below', 1),
 ('Project Gutenberg eBook, Details Below **', 1),
 ('** Please follow the copyright guidelines', 1),
 ('the copyright guidelines in this file.', 1)]

In [ ]:
countsDF = sqlContext.createDataFrame(countKeys) \
            .withColumnRenamed('_1','Word').withColumnRenamed('_2','Count')

countsDF.toPandas().to_csv('counts.csv', index=False)

countsDF.show()

+--------------------+-----+
|                Word|Count|
+--------------------+-----+
|EBook of The Comp...|    2|
|Complete Works of...|    1|
|eBook is for the ...|    2|
|for the use of an...|    2|
|the use of anyone...|    2|
|of anyone anywher...|    2|
|anyone anywhere a...|    2|
|You may copy it, ...|    2|
|may copy it, give...|    2|
|copy it, give it ...|    2|
|re-use it under t...|    2|
|it under the term...|    2|
|under the terms o...|    2|
|terms of the Proj...|    2|
|of the Project Gu...|    2|
|** This is a COPY...|    1|
|COPYRIGHTED Proje...|    1|
|Project Gutenberg...|    1|
|** Please follow ...|    1|
|the copyright gui...|    1|
+--------------------+-----+
only showing top 20 rows



In [ ]:
from google.colab import drive
drive.mount('/content/drive')